In [1]:
import pandas as pd
from transformers import pipeline
import re
import kss
import emoji
import numpy as np

/Users/apple/Softeer-DE/W2/M5/W2M5_team/w2team/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [5]:
from konlpy.tag import Mecab
mecab = Mecab()

In [2]:
df = pd.read_csv("reviews.csv")
df.head()

,지점명,닉네임,작성리뷰수,작성시간,방문일,방문횟수,인증수단,좋아요수,리뷰내용
0,신촌2호점,ok_****,466,25.11.27.목,점심에 방문,1번째 방문,영수증,0,멀리서 온 친구랑 술한잔 하고싶어서 방문했는데\n음식맛은 기본 한다는 더본 홍콩반점...
1,신촌2호점,rmdlll,31,25.12.2.화,저녁에 방문,1번째 방문,영수증,0,"저렴한 맛에 가는 곳 •••\n특별히 너무 맛있지는 않지만, 어떤 메뉴를 시켜도 맛..."
2,신촌2호점,재재슈슈,3,25.11.27.목,저녁에 방문,3번째 방문,영수증,0,신촌에서 친구랑 가볍게 한잔 하고싶어서 갈곳을 찾다가 딱걸린 홍콩반점.깐풍새우 소스...
3,신촌2호점,엉클뤼치,8,25.11.28.금,점심에 방문,4번째 방문,영수증,0,볶음밥 짱 맛있어요. 강추합니다\n직원분들도 친절하고 매장도 깨끗.\n재방문 예정 ...
4,신촌2호점,엉클뤼치,8,25.10.1.수,저녁에 방문,3번째 방문,영수증,0,정말 맛있게 잘 먹었습니다. 식당가격들이 다 올라서 힘든데 홍콩반점은 착한가격에 양...


In [3]:
df_review = df[['지점명', '리뷰내용']]
df_review.head(10)

,지점명,리뷰내용
0,신촌2호점,멀리서 온 친구랑 술한잔 하고싶어서 방문했는데\n음식맛은 기본 한다는 더본 홍콩반점...
1,신촌2호점,"저렴한 맛에 가는 곳 •••\n특별히 너무 맛있지는 않지만, 어떤 메뉴를 시켜도 맛..."
2,신촌2호점,신촌에서 친구랑 가볍게 한잔 하고싶어서 갈곳을 찾다가 딱걸린 홍콩반점.깐풍새우 소스...
3,신촌2호점,볶음밥 짱 맛있어요. 강추합니다\n직원분들도 친절하고 매장도 깨끗.\n재방문 예정 ...
4,신촌2호점,정말 맛있게 잘 먹었습니다. 식당가격들이 다 올라서 힘든데 홍콩반점은 착한가격에 양...
5,신촌2호점,탕짜면 맛있어요🖤
6,신촌2호점,날이 선선해지니 뜨거운 짬뽕국물이 그리워져서 혼밥하기 좋은 홍콩반점 다녀왔어요. 속...
7,신촌2호점,탕수육 정말 맛있어요~
8,신촌2호점,탕슈욱은 홍콩반점이 제일 맛있어요.~~ 튀김옷도 적당하고 색깔도 연하게 노릿노릿 한...
9,신촌2호점,음식 나오는 카운터에서 손톱 깎는 건 정말 아닌 것 같습니다. 위생적으로 굉장히 불...


In [4]:
def split_review_sentences(review):
    if not isinstance(review, str):
        return []
    
    # 1차: 개행 기준
    lines = [l.strip() for l in review.split('\n') if l.strip()]
    
    # 2차: 문장 분리
    sentences = []
    for line in lines:
        sentences.extend(kss.split_sentences(line))
    
    return sentences

def preprocess_review(review):
    sentences = split_review_sentences(review)
    sentences = [emoji.demojize(s) for s in sentences]
    return sentences

In [ ]:
sentiment_model = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment"
)

In [6]:
POSITIVE_WORDS = {
    "맛있": 2,
    "좋": 1.5,
    "만족": 2,
    "추천": 1.5,
    "친절": 1.5,
    "깔끔": 1,
    "재방문": 2
}

NEGATIVE_WORDS = {
    "최악": -3,
    "불친절": -2.5,
    "별로": -2,
    "불만족": -3,
    "실망": -2,
    "기분나쁘": -2,
    "다시는": -2
}

NEGATIONS = {"않", "못", "아니", "별로"}

In [19]:
def sentence_sentiment_score(sentence):
    morphs = mecab.morphs(sentence)
    score = 0
    print(morphs)

    for i, m in enumerate(morphs):
        # 긍정
        for w, s in POSITIVE_WORDS.items():
            if w in m:
                score += s
                print(f"pos: {w}")
                # 부정어 처리
                if i+1 < len(morphs) and morphs[i+1] in NEGATIONS:
                    print("but negate")
                    score -= s * 2

        # 부정
        for w, s in NEGATIVE_WORDS.items():
            if w in m:
                print(f"neg: {w}")
                score += s

    return score

def score_to_star(score):
    if score >= 3:
        return 5
    elif score >= 1.5:
        return 4
    elif score > -1:
        return 3
    elif score > -3:
        return 2
    else:
        return 1


def review_star_from_text_mecab(review):
    sentences = split_review_sentences(review)

    if len(sentences) == 0:
        return np.nan

    stars = []
    for sent in sentences:
        s = sentence_sentiment_score(sent)
        stars.append(score_to_star(s))

    return round(np.mean(stars), 2)

In [15]:
review = df_review.loc[25, '리뷰내용']
review

'짜장면이 너무 맛있어서 자주 방문하는 곳이에요! 다들 고기짜장에 고춧가루 뿌려드세요!!! 사장님도 너무 친절하셔서 좋아요~~☺️'

In [17]:
senten = preprocess_review(review)
senten

['짜장면이 너무 맛있어서 자주 방문하는 곳이에요!',
 '다들 고기짜장에 고춧가루 뿌려드세요!!!',
 '사장님도 너무 친절하셔서 좋아요~~:smiling_face:']

In [20]:
for s in senten:
    print(s)
    sentence_sentiment_score(s)

짜장면이 너무 맛있어서 자주 방문하는 곳이에요!
['짜장면', '이', '너무', '맛있', '어서', '자주', '방문', '하', '는', '곳', '이', '에요', '!']
pos: 맛있
다들 고기짜장에 고춧가루 뿌려드세요!!!
['다', '들', '고', '기', '짜', '장', '에', '고춧가루', '뿌려', '드세요', '!', '!!']
사장님도 너무 친절하셔서 좋아요~~:smiling_face:
['사장', '님', '도', '너무', '친절', '하', '셔서', '좋', '아요', '~~:', 'smiling', '_', 'face', ':']
pos: 친절
pos: 좋


In [21]:
sen = preprocess_review("짜장면은 별로 기대 안 했는데 맛있어요")
for s in sen:
    print(s)
    sentence_sentiment_score(s)

짜장면은 별로 기대 안 했는데 맛있어요
['짜장면', '은', '별로', '기대', '안', '했', '는데', '맛있', '어요']
neg: 별로
pos: 맛있


In [10]:
df_review["predicted_star"] = df_review["리뷰내용"].apply(review_star_from_text_mecab)

/var/folders/gy/mwkqsrr91251dr8cyvh7jhy00000gn/T/ipykernel_31140/4049552294.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review["predicted_star"] = df_review["리뷰내용"].apply(review_star_from_text_mecab)


In [11]:
df_review.head(30)

,지점명,리뷰내용,predicted_star
0,신촌2호점,멀리서 온 친구랑 술한잔 하고싶어서 방문했는데\n음식맛은 기본 한다는 더본 홍콩반점...,2.86
1,신촌2호점,"저렴한 맛에 가는 곳 •••\n특별히 너무 맛있지는 않지만, 어떤 메뉴를 시켜도 맛...",3.67
2,신촌2호점,신촌에서 친구랑 가볍게 한잔 하고싶어서 갈곳을 찾다가 딱걸린 홍콩반점.깐풍새우 소스...,3.50
3,신촌2호점,볶음밥 짱 맛있어요. 강추합니다\n직원분들도 친절하고 매장도 깨끗.\n재방문 예정 ...,3.50
4,신촌2호점,정말 맛있게 잘 먹었습니다. 식당가격들이 다 올라서 힘든데 홍콩반점은 착한가격에 양...,3.75
5,신촌2호점,탕짜면 맛있어요🖤,4.00
6,신촌2호점,날이 선선해지니 뜨거운 짬뽕국물이 그리워져서 혼밥하기 좋은 홍콩반점 다녀왔어요. 속...,3.67
7,신촌2호점,탕수육 정말 맛있어요~,4.00
8,신촌2호점,탕슈욱은 홍콩반점이 제일 맛있어요.~~ 튀김옷도 적당하고 색깔도 연하게 노릿노릿 한...,3.67
9,신촌2호점,음식 나오는 카운터에서 손톱 깎는 건 정말 아닌 것 같습니다. 위생적으로 굉장히 불...,3.00
